# Procesamiento de datos con Pandas en el subpaquete `teii.finance`

## Constructor de la clase derivada `TimesSeriesFinanceClient`

In [ ]:
import datetime
import json
import numpy as np
import pandas as pd
import requests
from importlib import resources

In [ ]:
pd.__version__

In [ ]:
_data_field2name_type = {
            "1. open":                  ("open",     "float"),
            "2. high":                  ("high",     "float"),
            "3. low":                   ("low",      "float"),
            "4. close":                 ("close",    "float"),
            "5. adjusted close":        ("aclose",   "float"),
            "6. volume":                ("volume",   "int"),
            "7. dividend amount":       ("dividend", "float"),
        }

In [ ]:
# Usamos IBM porque AMZN no repartió dividendos
query = 'https://www.alphavantage.co/query?function=TIME_SERIES_WEEKLY_ADJUSTED&symbol=IBM&apikey=MY_ALPHA_VANTAGE_API_KEY&data_type=json'

In [ ]:
download_data = False   # Poner a True si queremos descargarnos los datos de la web

In [ ]:
if download_data:
    # Realizamos la consulta
    response = requests.get(query)
    # print(response)
    # Extraemos los datos de la respuesta
    json_data_downloaded = response.json()
    # print(json.dumps(json_data_downloaded, indent=2))
    with open(f"TIME_SERIES_WEEKLY_ADJUSTED.IBM.json", 'w') as outfile:
            outfile.write(json.dumps(json_data_downloaded, indent=4))
else:
    with resources.open_text('teii.finance.data', 'TIME_SERIES_WEEKLY_ADJUSTED.IBM.json') as json_fid:
        json_data_downloaded = json.load(json_fid)

In [ ]:
json_metadata = json_data_downloaded['Meta Data']
print(json.dumps(json_metadata, indent=2))

In [ ]:
json_data = json_data_downloaded['Weekly Adjusted Time Series']
print(json.dumps(json_data, indent=2))

In [ ]:
# Build Panda's data frame
data_frame = pd.DataFrame.from_dict(json_data, orient='index', dtype=float)

In [ ]:
data_frame.describe()

In [ ]:
# Diccionario con las conversiones de nombre de las columnas
{key: name_type[0] for key, name_type in _data_field2name_type.items()}

In [ ]:
# Rename data fields
data_frame = data_frame.rename(columns={key: name_type[0] for key, name_type in _data_field2name_type.items()})

In [ ]:
data_frame.head()

In [ ]:
# Diccionario con las conversiones de tipos de las columnas
{name_type[0]: name_type[1] for key, name_type in _data_field2name_type.items()}

In [ ]:
# Set data field types
print(data_frame.dtypes)

data_frame = data_frame.astype(dtype={name_type[0]: name_type[1] for key, name_type in _data_field2name_type.items()})

data_frame.dtypes

In [ ]:
# Set index type to datetime
print(data_frame.index)

data_frame.index = data_frame.index.astype("datetime64[ns]")

data_frame.index

In [ ]:
# Sort data
print(data_frame.head())

data_frame = data_frame.sort_index(ascending=True)

data_frame.head()

A partir de aquí ya tenemos en `data_frame` los datos con los que vamos a trabajar, así que podemos usar este notebook para hacer las pruebas necesarias para extraer lo que nos piden en los métodos de `TimesSeriesFinanceClient()`.